In [1]:
from copy import deepcopy

In [79]:
def nussinov_algor(seq, MinLoop) :
    seq_size = len(seq)
    arr = [[0 for i in range(seq_size)] for j in range(seq_size)]
    paired = [('A','U'),('U','A'),('G','C'),('C','G')]
    for j in range(1,seq_size) :
        for i in range(j-1,-1,-1) :
            max_val = 0
            if j-i >= MinLoop :
                max_val = max(max_val,arr[i+1][j]) #i unpaired
                max_val = max(max_val,arr[i][j-1]) #j unparied
                if (seq[i],seq[j]) in paired :
                    max_val = max(max_val, arr[i+1][j-1] + 1)
                for k in range(i+1,j) :
                    max_val = max(max_val,arr[i][k] + arr[k+1][j])
            arr[i][j] = max_val
    return arr

def traceback(arr, seq, i, j) :
    seq_size = len(seq)
    paired = [('A','U'),('U','A'),('G','C'),('C','G')]
    if (seq[i],seq[j]) in paired and arr[i+1][j-1]+1 == arr[i][j] :
        return '('+ traceback(arr,seq,i+1,j-1) + ')'  #prioritize stem
    if  i > j :
        return ''
    if arr[i][j] == arr[i+1][j] :
        return '.' + traceback(arr, seq, i+1, j)
    if arr[i][j] == arr[i][j-1] :
        return traceback(arr, seq, i, j-1) + '.'
    for k in range(i+1, j) :
        if arr[i][k] + arr[k+1][j] == arr[i][j] :
            return traceback(arr,seq,i,k) + traceback(arr,seq,k+1,j)
    
def nussinov(seq, MinLoop = 4) :
    i = 0
    j = len(seq)-1
    arr = nussinov_algor(seq,MinLoop)
    return traceback(arr,seq,i,j)

In [85]:
def init(W,V,n):
    for l in range (-1 ,4):
        for i in range(0, n-l):
            W[i][i+l] = 0
            V[i][i+l] = -float('infinity')
    return (W,V)

comp = [('A','U'),('U','A'),('G','C'),('C','G'),('U','G')]

def forward(W, V, seq, n):
    for l in range(4, n):
        for i in range(0,n-l):
            if (seq[i],seq[i+l]) in comp:
                V[i][i+l] = max(W[i+1][i+l-1], 1+V[i+1][i+l-1])
            else :
                V[i][i+l] = -float('infinity')
            W[i][i+l] = W[i+1][i+l]
            tmpMax = 0
            for k in range(i+4, i+l+1):
                tmpMax = max(tmpMax, V[i][k]+W[k+1][i+l])
            tmpMax  = max(tmpMax, V[i][i+l])
            W[i][i+l] = max(W[i+1][i+l], tmpMax)
    return (W, V)

def traceVpath(i, j, W, V):
    if V[i][j]==W[i+1][j-1]:
        traceWpath(i+1, j-1, W, V)
    elif V[i][j]==1+V[i+1][j-1]:
        traceVpath(i+1, j-1, W, V)
    if i not in structure and j not in structure:
        structure[i] = j
        structure[j] = i
        
def traceWpath(i, j, W, V):
    if i < j and W[i][j] == W[i+1][j]:
        traceWpath(i+1, j, W, V)
    else:
        if i < j:
            for k in range(i+3, j+1):
                if W[i][j] == V[i][k] + W[k+1][j]:
                    traceWpath(k+1, j, W, V)
                    traceVpath(i, k, W, V)
                    break
                    
def CustomFold(seq):
    n = len(seq)
    global structure
    structure = [None for i in range(0, n)]
    W = [[None for c in range(1,n+3)] for d in range(1,n+3)]
    V = [[None for c in range(1,n+3)] for d in range(1,n+3)]
    initW,initV = init(W, V, n)
    filledW,filledV = forward(initW,initV,seq,n)
    traceWpath(0,n-1,filledW,filledV)
    annot = ''.join(['.' if j is None else ')' if j < i else '(' for i, j in enumerate(structure)])
    return annot
                    
                    

In [17]:
seq = "_NNNUUGGUGGCGAUAGCGAAGAGGUCACACCCGUUCCCAUACCGAACACGGAAGUUAAGCUCUUCAGCGCCGAUGGUAGUCGGGGGUUUCCCCCUGUGAGAGUAGGACGCCGCCAAGC"
print(len(seq))

119


In [78]:
def calculateConsecPairs(rna,MinStem,MinLoop):
    judge = [('A','U'),('U','A'),('G','C'),('C','G'),('G','U'),('U','G')]
    n = len(rna)-1
    pairs = []
    for i in range(1, n-2*MinStem-MinLoop+1):
        for j in range(i+2*MinStem+MinLoop-1, n+1):
            conPair = 0
            k = 0
            while k <= ((j-i-MinLoop+1)/2):
                if (rna[i+k],rna[j-k]) in judge:
                    conPair += 1
                    if conPair >= MinStem:
                        tempPair = (i,j,conPair)
                        pairs.append(tempPair)
                else:
                    break
                k += 1
    pairs.sort(key = lambda x : -(x[1]-x[0]))
    return pairs

def getPair(seq) :  
    k = 20
    pairList = []

    while True:
        pairList = calculateConsecPairs(seq, k, 3)
        if len(pairList)>100:
            pairList = calculateConsecPairs(seq,k+1,3) #editted
            break
        k-=1
    ret = []
    for (start, end, length) in pairList :
        ret += [Node(start,end,length)]
    
    return ret

In [13]:
class State :
    def __init__(self,seq) :
        self.totalStem = 0 
        self.nodes = []
        self.seqLen = len(seq)
        self.currentState = 0
        
    def insert(self, node) :
        self.nodes.append(node)
        self.totalStem += node.len
        
    def __lt__(self,other) :
        return self.currentState < other.currentState
    
    def __eq__(self,other) :
        return self.totalStem == other.totalStem
    
    def __gt__(self,other) :
        return self.totalStem > other.totalStem
    
    def getAnswer(self) :
        return [(node.start,node.end,node.len) for node in self.nodes]
        
        
class Node :
    def __init__(self, start, end, length) :
        self.start = start
        self.end = end
        self.len = length
        
    def __str__(self) :
        return f'{self.start}, {self.end}, {self.len}'
        
class PriorityQueue(object): 
    def __init__(self): 
        self.queue = [] 

    # for checking if the queue is empty 
    def isEmpty(self): 
        return len(self.queue) == 0
  
    # for inserting an element in the queue 
    def push(self, data): 
        self.queue.append(data) 
  
    # for popping an element based on Priority 
    def pop(self): 
        try: 
            currentMax = 0
            for i in range(len(self.queue)): 
                if self.queue[i] < self.queue[currentMax]: 
                    currentMax = i 
            item = self.queue[currentMax] 
            del self.queue[currentMax] 
            return item 
        
        except IndexError: 
            print() 
            exit() 
        
def checkConfilct(state, selectedNode) :
    frontS = selectedNode.start
    backS = selectedNode.end
    
    for node in state.nodes :
        front = node.start+node.len
        back = node.end- node.len
        if (frontS > front and backS < back) or (frontS > back+node.len) or (backS < front-node.len) :
            continue
        else:
            return False        
        
    return True

In [45]:
def run(seq) :
    state = State(seq)
    pq = PriorityQueue()
    pairList = getPair(seq)
    
    state1 = State(seq)   
    state1.insert(pairList[0])
    state1.currentState+=1
    
    state2 = State(seq)
    state2.currentState += 1
    
    pq.push(state1)
    pq.push(state2)
    
    ans = State(seq)
    
    while(not pq.isEmpty()) :
        state = pq.pop()
        cur = state.currentState
        
        print(f'current = {cur}, state = {state.getAnswer()}, total = {state.totalStem}')
        
        if cur == len(pairList)-1 :
            if ans.totalStem < state.totalStem :
                ans = state
            continue 

        cur += 1
        
        state1 = deepcopy(state)
        state1.currentState = cur
        pq.push(state1)
        
        state2 = deepcopy(state)
        if (checkConfilct(state2,pairList[cur])):
            state2.currentState = cur
            state2.insert(pairList[cur])
            pq.push(state2)
    return ans

In [46]:
ans = run(seq)
ans.getAnswer()

current = 1, state = [(4, 116, 5)], total = 5
current = 1, state = [], total = 0
current = 2, state = [(4, 116, 5)], total = 5
current = 2, state = [], total = 0
current = 2, state = [(4, 116, 7)], total = 7
current = 3, state = [(4, 116, 5)], total = 5
current = 3, state = [], total = 0
current = 3, state = [(4, 116, 8)], total = 8
current = 3, state = [(4, 116, 7)], total = 7
current = 4, state = [(4, 116, 5)], total = 5
current = 4, state = [], total = 0
current = 4, state = [(4, 116, 9)], total = 9
current = 4, state = [(4, 116, 8)], total = 8
current = 4, state = [(4, 116, 7)], total = 7
current = 5, state = [(4, 116, 5)], total = 5
current = 5, state = [], total = 0
current = 5, state = [(5, 115, 5)], total = 5
current = 5, state = [(4, 116, 9)], total = 9
current = 5, state = [(4, 116, 8)], total = 8
current = 5, state = [(4, 116, 7)], total = 7
current = 6, state = [(4, 116, 5)], total = 5
current = 6, state = [], total = 0
current = 6, state = [(5, 115, 6)], total = 6
current 

current = 31, state = [(6, 114, 6), (25, 99, 5), (34, 86, 5)], total = 16
current = 31, state = [(6, 114, 6), (25, 99, 5), (33, 87, 6)], total = 17
current = 31, state = [(6, 114, 6), (25, 99, 5), (33, 87, 5)], total = 16
current = 31, state = [(6, 114, 5)], total = 5
current = 31, state = [(6, 114, 5), (47, 98, 5)], total = 10
current = 31, state = [(6, 114, 5), (34, 86, 5)], total = 10
current = 31, state = [(6, 114, 5), (33, 87, 6)], total = 11
current = 31, state = [(6, 114, 5), (33, 87, 5)], total = 10
current = 31, state = [(6, 114, 5), (22, 89, 5)], total = 10
current = 31, state = [(6, 114, 5), (21, 90, 6)], total = 11
current = 31, state = [(6, 114, 5), (21, 90, 5)], total = 10
current = 31, state = [(6, 114, 5), (25, 99, 5)], total = 10
current = 31, state = [(6, 114, 5), (25, 99, 5), (34, 86, 5)], total = 15
current = 31, state = [(6, 114, 5), (25, 99, 5), (33, 87, 6)], total = 16
current = 31, state = [(6, 114, 5), (25, 99, 5), (33, 87, 5)], total = 15
current = 31, state =

current = 35, state = [(7, 113, 5), (34, 86, 5)], total = 10
current = 35, state = [(7, 113, 5), (33, 87, 6)], total = 11
current = 35, state = [(7, 113, 5), (33, 87, 5)], total = 10
current = 35, state = [(7, 113, 5), (22, 89, 5)], total = 10
current = 35, state = [(7, 113, 5), (22, 89, 5), (31, 76, 5)], total = 15
current = 35, state = [(7, 113, 5), (21, 90, 6)], total = 11
current = 35, state = [(7, 113, 5), (21, 90, 6), (31, 76, 5)], total = 16
current = 35, state = [(7, 113, 5), (21, 90, 5)], total = 10
current = 35, state = [(7, 113, 5), (21, 90, 5), (31, 76, 5)], total = 15
current = 35, state = [(7, 113, 5), (25, 99, 5)], total = 10
current = 35, state = [(7, 113, 5), (25, 99, 5), (31, 76, 5)], total = 15
current = 35, state = [(7, 113, 5), (25, 99, 5), (34, 86, 5)], total = 15
current = 35, state = [(7, 113, 5), (25, 99, 5), (33, 87, 6)], total = 16
current = 35, state = [(7, 113, 5), (25, 99, 5), (33, 87, 5)], total = 15
current = 35, state = [(8, 115, 5)], total = 5
current 

current = 37, state = [(6, 114, 5), (34, 86, 5)], total = 10
current = 37, state = [(6, 114, 5), (33, 87, 6)], total = 11
current = 37, state = [(6, 114, 5), (33, 87, 5)], total = 10
current = 37, state = [(6, 114, 5), (22, 89, 5)], total = 10
current = 37, state = [(6, 114, 5), (22, 89, 5), (31, 76, 5)], total = 15
current = 37, state = [(6, 114, 5), (21, 90, 6)], total = 11
current = 37, state = [(6, 114, 5), (21, 90, 6), (31, 76, 5)], total = 16
current = 37, state = [(6, 114, 5), (21, 90, 5)], total = 10
current = 37, state = [(6, 114, 5), (21, 90, 5), (31, 76, 5)], total = 15
current = 37, state = [(6, 114, 5), (25, 99, 5)], total = 10
current = 37, state = [(6, 114, 5), (25, 99, 5), (31, 76, 5)], total = 15
current = 37, state = [(6, 114, 5), (25, 99, 5), (34, 86, 5)], total = 15
current = 37, state = [(6, 114, 5), (25, 99, 5), (33, 87, 6)], total = 16
current = 37, state = [(6, 114, 5), (25, 99, 5), (33, 87, 5)], total = 15
current = 37, state = [(5, 115, 8)], total = 8
current 

current = 39, state = [(4, 116, 5), (21, 90, 6), (31, 76, 5)], total = 16
current = 39, state = [(4, 116, 5), (21, 90, 5)], total = 10
current = 39, state = [(4, 116, 5), (21, 90, 5), (31, 76, 5)], total = 15
current = 39, state = [(4, 116, 5), (25, 99, 5)], total = 10
current = 39, state = [(4, 116, 5), (25, 99, 5), (31, 76, 5)], total = 15
current = 39, state = [(4, 116, 5), (25, 99, 5), (34, 86, 5)], total = 15
current = 39, state = [(4, 116, 5), (25, 99, 5), (33, 87, 6)], total = 16
current = 39, state = [(4, 116, 5), (25, 99, 5), (33, 87, 5)], total = 15
current = 39, state = [(4, 116, 5), (32, 109, 5)], total = 10
current = 39, state = [(4, 116, 5), (32, 109, 5), (60, 102, 5)], total = 15
current = 39, state = [(4, 116, 5), (32, 109, 5), (59, 103, 6)], total = 16
current = 39, state = [(4, 116, 5), (32, 109, 5), (59, 103, 5)], total = 15
current = 39, state = [(4, 116, 5), (32, 109, 5), (47, 98, 5)], total = 15
current = 39, state = [], total = 0
current = 39, state = [(4, 45, 5)

...

current = 67, state = [(6, 114, 7), (47, 98, 5), (59, 87, 6)], total = 18
current = 67, state = [(6, 114, 7), (47, 98, 5), (59, 87, 5)], total = 17
current = 67, state = [(6, 114, 7), (47, 98, 5), (58, 88, 7)], total = 19
current = 67, state = [(6, 114, 7), (47, 98, 5), (58, 88, 6)], total = 18
current = 67, state = [(6, 114, 7), (47, 98, 5), (58, 88, 5)], total = 17
current = 67, state = [(6, 114, 7), (47, 98, 5), (57, 89, 8)], total = 20
current = 67, state = [(6, 114, 7), (47, 98, 5), (57, 89, 7)], total = 19
current = 67, state = [(6, 114, 7), (47, 98, 5), (57, 89, 6)], total = 18
current = 67, state = [(6, 114, 7), (47, 98, 5), (57, 89, 5)], total = 17
current = 67, state = [(6, 114, 7), (47, 98, 5), (57, 89, 5), (70, 83, 5)], total = 22
current = 67, state = [(6, 114, 7), (34, 86, 5)], total = 12
current = 67, state = [(6, 114, 7), (34, 86, 5), (93, 106, 5)], total = 17
current = 67, state = [(6, 114, 7), (34, 86, 5), (40, 78, 5)], total = 17
current = 67, state = [(6, 114, 7), (

current = 67, state = [(6, 114, 5), (21, 90, 6), (31, 76, 5), (93, 106, 5)], total = 21
current = 67, state = [(6, 114, 5), (21, 90, 5)], total = 10
current = 67, state = [(6, 114, 5), (21, 90, 5), (93, 106, 5)], total = 15
current = 67, state = [(6, 114, 5), (21, 90, 5), (70, 83, 5)], total = 15
current = 67, state = [(6, 114, 5), (21, 90, 5), (70, 83, 5), (93, 106, 5)], total = 20
current = 67, state = [(6, 114, 5), (21, 90, 5), (40, 78, 5)], total = 15
current = 67, state = [(6, 114, 5), (21, 90, 5), (40, 78, 5), (93, 106, 5)], total = 20
current = 67, state = [(6, 114, 5), (21, 90, 5), (31, 76, 5)], total = 15
current = 67, state = [(6, 114, 5), (21, 90, 5), (31, 76, 5), (93, 106, 5)], total = 20
current = 67, state = [(6, 114, 5), (25, 99, 5)], total = 10
current = 67, state = [(6, 114, 5), (25, 99, 5), (70, 83, 5)], total = 15
current = 67, state = [(6, 114, 5), (25, 99, 5), (61, 86, 5)], total = 15
current = 67, state = [(6, 114, 5), (25, 99, 5), (60, 86, 5)], total = 15
current

current = 67, state = [(5, 115, 6), (60, 102, 5), (82, 94, 5)], total = 16
current = 67, state = [(5, 115, 6), (60, 102, 5), (82, 95, 5)], total = 16
current = 67, state = [(5, 115, 6), (60, 102, 5), (70, 83, 5)], total = 16
current = 67, state = [(5, 115, 6), (60, 102, 5), (81, 96, 6)], total = 17
current = 67, state = [(5, 115, 6), (60, 102, 5), (81, 96, 5)], total = 16
current = 67, state = [(5, 115, 6), (59, 103, 6)], total = 12
current = 67, state = [(5, 115, 6), (59, 103, 6), (82, 94, 5)], total = 17
current = 67, state = [(5, 115, 6), (59, 103, 6), (82, 95, 5)], total = 17
current = 67, state = [(5, 115, 6), (59, 103, 6), (70, 83, 5)], total = 17
current = 67, state = [(5, 115, 6), (59, 103, 6), (81, 96, 6)], total = 18
current = 67, state = [(5, 115, 6), (59, 103, 6), (81, 96, 5)], total = 17
current = 67, state = [(5, 115, 6), (59, 103, 5)], total = 11
current = 67, state = [(5, 115, 6), (59, 103, 5), (82, 94, 5)], total = 16
current = 67, state = [(5, 115, 6), (59, 103, 5), (

current = 67, state = [(4, 116, 7), (47, 98, 5)], total = 12
current = 67, state = [(4, 116, 7), (47, 98, 5), (70, 83, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (61, 86, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (60, 86, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (59, 87, 6)], total = 18
current = 67, state = [(4, 116, 7), (47, 98, 5), (59, 87, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (58, 88, 7)], total = 19
current = 67, state = [(4, 116, 7), (47, 98, 5), (58, 88, 6)], total = 18
current = 67, state = [(4, 116, 7), (47, 98, 5), (58, 88, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (57, 89, 8)], total = 20
current = 67, state = [(4, 116, 7), (47, 98, 5), (57, 89, 7)], total = 19
current = 67, state = [(4, 116, 7), (47, 98, 5), (57, 89, 6)], total = 18
current = 67, state = [(4, 116, 7), (47, 98, 5), (57, 89, 5)], total = 17
current = 67, state = [(4, 116, 7), (47, 98, 5), (5

[(4, 116, 9), (50, 91, 7), (70, 83, 5), (93, 106, 5)]

In [47]:
ans.seqLen

119

In [75]:
def cleaning(state,seq,algor) :
    out = ['' for i in range(len(seq))]
    unavailable = []
    for start,end,length in state.getAnswer() :
        unavailable += [[start,start+length-1]]
        unavailable += [[end-length+1,end]]
        for i in range(length) :
            out[start+i] = '('
            out[end-i] = ')'
    unavailable.sort(key = lambda x : x[0])
    available = []
    idx = 1
    for start,end in unavailable :
        available += [[idx,start-1]]
        idx = end+1
    available += [[idx,len(seq)-1]]
    
    for start,end in available :
        if end-start > 3:
            tempSeq = seq[start:end+1]
            tempDot = algor(tempSeq)
            for i in range(len(tempDot)) :
                out[start+i] = tempDot[i]
        else :
            for i in range(start,end+1) :
                out[i] = '.'
    return ''.join(out)
    
    
    
# input : 100,  [[20,50], [60,90]]
# output : [[0,19],[51,59],[91,100]]

In [86]:
cleaning(ans,seq,nussinov)

'...(((((((((.(.(((..(.(((...))))(((...).))))..)).(((((((..(((....))).(((((....))))).))))))).(((((....))))).)))))))))..'

In [87]:
cleaning(ans,seq,CustomFold)

'...(((((((((..........(((...))).((((.......))))..(((((((..(((....))).(((((....))))).))))))).(((((....))))).)))))))))..'

In [88]:
nussinov(seq[1:])

'...((((.((((.(..(((((.(((...)))).)))(..((((...(((.((((..(...)))))((.((((((...).))))(((...))))))))).).)))))))))).))))..'

In [89]:
CustomFold(seq[1:])

'...(((((((((....))....(((...)))(((((..((.(((....)))..))((..((((((((...((((....))))((((...))))))).)))))))))))))))))))..'